In [ ]:
####################################################
## script created by Will Deadman and Thomas Huet ##
####################################################

## Satellite imagery and Heritage Place geometry

Plot EAMENA Heritage Places polygon geometries using Earth Engine (`ee`) and use theses geometries to clip aerial photographs (ex: LANDSAT)

Load libraries

In [2]:
import ee
import requests
import json
import folium
from IPython.core.display import display, HTML

Google credentials to access GEE

In [3]:
ee.Authenticate()
ee.Initialize(project = "ee-zoometh")

Load the geomtries

In [4]:
url = "https://raw.githubusercontent.com/eamena-project/eamena-arches-dev/main/projects/caravanserail/caravanserail.geojson"
response = requests.get(url)
geojson = json.loads(response.text)

Geometries used to clip

In [5]:
my_geom = 3 # select a geometry (1, 3, etc.) /!\ doesn't work with POINTS
geom_type = geojson['features'][my_geom]['geometry']['type']

if geom_type == 'Polygon':
  EAMENA_ID = geojson['features'][my_geom]['properties']['EAMENA ID']

  # coordinates
  hp_coords = geojson['features'][my_geom]['geometry']['coordinates']
  hp = ee.Geometry.Polygon(hp_coords)

  # centroid
  centroid = hp.centroid()
  centroid_info = centroid.getInfo()
  coord_x = centroid_info['coordinates'][0]
  coord_y = centroid_info['coordinates'][1]

  # aoi = bbox/mbr
  flat_coords = [item for sublist in hp_coords for item in sublist]
  min_lon, min_lat = min(flat_coords, key=lambda x: x[0])[0], min(flat_coords, key=lambda x: x[1])[1]
  max_lon, max_lat = max(flat_coords, key=lambda x: x[0])[0], max(flat_coords, key=lambda x: x[1])[1]
  AOI = ee.Geometry.BBox(min_lon, min_lat, max_lon, max_lat) # bbox

  # date
  Y1 = 2021
  Y2 = 2021
  M1 = 7
  M2 = M1 + 3

  # Sentinel-2 composite
  collection = (
      ee.ImageCollection("COPERNICUS/S2_SR")
      .filterBounds(AOI)
      .filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than", 10)
      .filter(ee.Filter.calendarRange(Y1, Y2, "year"))
      .filter(ee.Filter.calendarRange(M1, M2, "month"))
      .select(["B.*"])
  )

  image = ee.Image(collection.median()).clip(AOI).divide(10000)
  print("Done")
else:
  print("The current geometry is " + geom_type)

Done


Show Map

In [11]:
# type(hp)


[[[52.179277, 34.759593],
  [52.179776, 34.75943],
  [52.179594, 34.758844],
  [52.179025, 34.759055],
  [52.179277, 34.759593]]]

In [14]:
# Visualization parameters.
vis_params = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

# Folium map centered
map = folium.Map(location=[coord_y, coord_x], zoom_start=18, tiles=None)  # Set tiles=None to start with a blank map

# Google Satellite
google_satellite = folium.TileLayer(
    tiles='https://{s}.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Satellite',
    max_zoom=20,
    subdomains=['mt1', 'mt2', 'mt3'],
    overlay=False,
    control=True,
).add_to(map)

# HP shape
hp_geojson = ee.FeatureCollection([ee.Feature(hp)]).getInfo()
folium.GeoJson(
    data=hp_geojson,
    style_function=lambda x: {
        'color': 'blue',
        'weight': 1,
        'fillOpacity': 0,
    },
    name='HP Geometry'
).add_to(map)

# clipped geometry
map_id_dict = ee.Image(image).getMapId(vis_params)
folium.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    control=True,
    name='median composite',
).add_to(map)

# control panel
map.add_child(folium.LayerControl())

# Display the map.
map_html = map._repr_html_()
map_display = HTML(data=f'<div style="width: 800px; height: 800px;">{map_html}</div>')
display(map_display)


Export

In [ ]:
# export
task = ee.batch.Export.image.toDrive(
    image=image,
    description = EAMENA_ID,
    folder="gee_python",
    region=AOI,
    scale=100,
    crs="EPSG:4326",
)

task.start()

In [ ]:
task.status()

{'state': 'COMPLETED',
 'description': 'EAMENA-0164906',
 'creation_timestamp_ms': 1710155457515,
 'update_timestamp_ms': 1710155483587,
 'start_timestamp_ms': 1710155463673,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/#folders/1hxwSqjtmcAk2baSX0nvbW7ZT6OmuUEIn'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 1.1529306173324585,
 'id': 'N2ZUYMN3EEHOPXYAQFBDMAET',
 'name': 'projects/ee-zoometh/operations/N2ZUYMN3EEHOPXYAQFBDMAET'}